### sdxl-wrong-lora Testing

A Colab Notebook to test out the impact of [sdxl-wrong-lora](https://huggingface.co/minimaxir/sdxl-wrong-lora) vs. the output when not using the LoRA.

In [ ]:
!pip install -q transformers accelerate git+https://github.com/huggingface/diffusers compel

In [ ]:
from diffusers import DiffusionPipeline
import torch
from compel import Compel, ReturnedEmbeddingsType
from diffusers import DiffusionPipeline, AutoencoderKL
from PIL import Image
from datetime import datetime
from tqdm.auto import tqdm
import os
from random import randint

vae = AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix",
                                    torch_dtype=torch.float16)
base = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    vae=vae,
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True
)

base.load_lora_weights("minimaxir/sdxl-wrong-lora")

_ = base.to("cuda")

refiner = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-refiner-1.0",
    text_encoder_2=base.text_encoder_2,
    vae=base.vae,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
)

_ = refiner.to("cuda")

In [ ]:
def image_grid(imgs, rows, cols):
    assert len(imgs) == rows * cols

    w, h = imgs[0].size
    grid = Image.new("RGB", size=(cols * w, rows * h))
    grid_w, grid_h = grid.size

    for i, img in enumerate(imgs):
        grid.paste(img, box=(i % cols * w, i // cols * h))
    return grid

compel_base = Compel(tokenizer=[base.tokenizer, base.tokenizer_2] , text_encoder=[base.text_encoder, base.text_encoder_2], returned_embeddings_type=ReturnedEmbeddingsType.PENULTIMATE_HIDDEN_STATES_NON_NORMALIZED, requires_pooled=[False, True])
compel_refiner = Compel(tokenizer=refiner.tokenizer_2 , text_encoder=refiner.text_encoder_2, returned_embeddings_type=ReturnedEmbeddingsType.PENULTIMATE_HIDDEN_STATES_NON_NORMALIZED, requires_pooled=True)

high_noise_frac = 0.8
base.set_progress_bar_config(disable=True)
refiner.set_progress_bar_config(disable=True)

In [ ]:
def wrong_grid(source_prompt, cfg=13, seed=42):
    all_images = []

    states = [(source_prompt, None, 0.),
              (source_prompt, "wrong", 1.),
             ]

    if seed < 0:
        seed = randint(0, 10**8)
        print(f"Seed: {seed}")

    for state in tqdm(states):
        prompt = state[0]
        negative_prompt = state[1]
        lora_weight = state[2]

        conditioning, pooled = compel_base(prompt)
        conditioning_neg, pooled_neg = compel_base(negative_prompt) if negative_prompt is not None else (None, None)
        generator = torch.Generator(device="cuda").manual_seed(seed)

        latents = base(prompt_embeds=conditioning,
                    pooled_prompt_embeds=pooled,
                    negative_prompt_embeds=conditioning_neg,
                    negative_pooled_prompt_embeds=pooled_neg,
                    guidance_scale=cfg,
                    denoising_end=high_noise_frac,
                    generator=generator,
                    output_type="latent",
                    cross_attention_kwargs={"scale": lora_weight}).images

        conditioning, pooled = compel_refiner(prompt)
        conditioning_neg, pooled_neg = compel_refiner(negative_prompt) if negative_prompt is not None else (None, None)
        generator = torch.Generator(device="cuda").manual_seed(seed)

        images = refiner(
            prompt_embeds=conditioning,
            pooled_prompt_embeds=pooled,
            negative_prompt_embeds=conditioning_neg,
            negative_pooled_prompt_embeds=pooled_neg,
            guidance_scale=cfg,
            denoising_start=high_noise_frac,
            image=latents,
            generator=generator,
            ).images

        all_images.append(images[0])

    image_large = image_grid(all_images, 1, 2)
    display(image_large.resize((image_large.width // 2, image_large.height // 2)))
    image_large.save("grid.png")

    output_folder = datetime.now().strftime('%Y%m%d_%H%M%S')

    if not os.path.exists("outputs"):
        os.mkdir("outputs")

    if not os.path.exists(os.path.join("outputs", output_folder)):
        os.mkdir(os.path.join("outputs", output_folder))

    for i, image in enumerate(all_images):
        image.save(os.path.join("outputs", output_folder, f"{i+1:02d}.png"))

## Test it out!

This notebook implements the compel syntax, which allows you to weight groups of terms in the prompt.

- `prompt`: The base prompt to test.
- `cfg`: The classifier-free guidance. Normally SDXL has a default of `7.5` but I find a high one like `13` works better with SDXL.
- `seed`: The seed for both image generations. Setting it to `-1` will choose a random seed and print it for future reference. You can provide a seed for reproducible generations, although the results may not be 100% identical due to general pseudorandomness.

You can download the pair of images at full resolution from the sidebar as `grid.png`. You can find the individual image outputs inside the `outputs` folder.

In [ ]:
prompt = "hyperrealistic death metal album cover featuring edgy moody realistic (human Super Mario)++, edgy and moody"
cfg = 13
seed = -1

wrong_grid(prompt, cfg, seed)

## MIT License

Copyright (c) 2023 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.